# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686556


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:17,  2.75s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:52,  1.96s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:48,  1.87s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:33,  1.36s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  4.15it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  4.21it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  4.27it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  4.07it/s]

not-logged-in-3e461b5a5635ce55ae0f    0.003460
not-logged-in-f6fac0763296d5759c01    0.005735
not-logged-in-68d3d82149e88f995533    1.098097
Dynamatt                              0.012432
not-logged-in-f7e2260ed4a9517a3e4a    0.001911
cschwefl                              0.018830
Camelron                              0.000894
Claudiux                              0.002119
not-logged-in-a7f3c0eb6d81d69e04a2    0.001120
not-logged-in-6e654a118e00ea29ed67    0.002103
not-logged-in-9a45911eb71184fe7c7f    0.002028
not-logged-in-324fceed15c9d89c092f    0.001034
not-logged-in-24db7619b192ada33b8d    0.028230
homant                                0.000925
41Letters                             0.001047
acapirala                             0.001065
BraxtonKinney                         0.001141
not-logged-in-3569dcff4dbd0ee90f50    0.002195
Lavadude                              0.002860
jcm7                                  0.001930
ShirleyL                              0.001756
Russell_Hoch 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())